In [8]:
import cv2
import mediapipe as mp
import numpy as np
import time, os, csv, serial

# MediaPipe 설정
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 시리얼 포트 설정
ser = serial.Serial('/dev/ttyACM3', 9600)

# 카메라 설정
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

# 동작 및 시간 설정
actions = ['upper_left', 'lower_left']
seq_length = 30
secs_for_action = 60
created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

# 데이터 수집 및 저장
while cap.isOpened():
    for idx, action in enumerate(actions):
        combined_data = []  # 미디어파이프 데이터와 센서 데이터를 결합한 리스트

        ret, img = cap.read()
        img = cv2.flip(img, 1)
        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(8000)

        start_time = time.time()

        # 센서 데이터 초기값 설정
        last_gyro_x = last_gyro_y = last_gyro_z = last_acc_x = last_acc_y = last_acc_z = 0
        gyro_x = gyro_y = gyro_z = acc_x = acc_y = acc_z = 0

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()
            img = cv2.flip(img, 1)
            out.write(img)  # 영상 저장

            # MediaPipe 처리
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # MediaPipe 데이터 처리
            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # 계산된 각도
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
                    v = v2 - v1
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
                    angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
                    angle = np.degrees(angle)

                    # 미디어파이프 데이터와 라벨
                    mediapipe_data_only = np.concatenate([joint.flatten(), angle])
                    action_label = idx

                    # 센서 데이터 처리
                    new_data_collected = False
                    if ser.in_waiting > 0:
                        ser_data = ser.readline().decode('utf-8').strip()

                        try:
                            if 'Gyroscope values' in ser_data:
                                gyro_values = ser_data.split(': ')[1].split(', ')
                                gyro_x, gyro_y, gyro_z = [float(val) for val in gyro_values]
                                new_data_collected = True

                            elif 'Acceleration values' in ser_data:
                                acc_values = ser_data.split(': ')[1].split(', ')
                                acc_x, acc_y, acc_z = [float(val) for val in acc_values]
                                new_data_collected = True

                        except ValueError as e:
                            print("Sensor data parsing error:", e)

                    # 센서 데이터가 수집되지 않았다면 이전 값을 사용
                    if not new_data_collected:
                        gyro_x, gyro_y, gyro_z = last_gyro_x, last_gyro_y, last_gyro_z
                        acc_x, acc_y, acc_z = last_acc_x, last_acc_y, last_acc_z

                    # 데이터 결합 및 저장
                    sensor_data = [gyro_x, gyro_y, gyro_z, acc_x, acc_y, acc_z]
                    combined_row = np.concatenate((mediapipe_data_only, sensor_data, [action_label]))
                    combined_data.append(combined_row)

                    # 마지막 센서값 갱신
                    last_gyro_x, last_gyro_y, last_gyro_z = gyro_x, gyro_y, gyro_z
                    last_acc_x, last_acc_y, last_acc_z = acc_x, acc_y, acc_z

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        # 결합된 데이터 저장
        combined_data = np.array(combined_data)
        np.save(os.path.join('dataset', f'combined_{action}_{created_time}'), combined_data)

        # 시퀀스 데이터 생성 및 저장
        full_seq_data = []
        for seq in range(len(combined_data) - seq_length):
            full_seq_data.append(combined_data[seq:seq + seq_length])
        full_seq_data = np.array(full_seq_data)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)

    break

cap.release()
out.release()
cv2.destroyAllWindows()


I0000 00:00:1702802951.690338   17468 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702802951.717787   20785 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.54.03), renderer: NVIDIA RTX A5000/PCIe/SSE2
QObject::moveToThread: Current thread (0x5f629d0) is not the object's thread (0x68aff40).
Cannot move to target thread (0x5f629d0)

QObject::moveToThread: Current thread (0x5f629d0) is not the object's thread (0x68aff40).
Cannot move to target thread (0x5f629d0)

QObject::moveToThread: Current thread (0x5f629d0) is not the object's thread (0x68aff40).
Cannot move to target thread (0x5f629d0)

QObject::moveToThread: Current thread (0x5f629d0) is not the object's thread (0x68aff40).
Cannot move to target thread (0x5f629d0)

QObject::moveToThread: Current thread (0x5f629d0) is not the object's thread (0x68aff40).
Cannot move to target thread (0x5f629d0)

QObject::moveToThread: Current thread (0x5f629d0) is not the object's thread (0x68aff40

Sensor data parsing error: could not convert string to float: 'Acceleration values (x'
Sensor data parsing error: could not convert string to float: '0.67Acceleration values (x'
Sensor data parsing error: could not convert string to float: '-35.2Acceleration values (x'
Sensor data parsing error: could not convert string to float: '-10.07Acceleration values (x'
Sensor data parsing error: could not convert string to float: '-Acceleration values (x'
Sensor data parsing error: could not convert string to float: '41.5Acceleration values (x'
Sensor data parsing error: could not convert string to float: '-0.Acceleration values (x'
Sensor data parsing error: could not convert string to float: '-14.28Acceleration values (x'
Sensor data parsing error: could not convert string to float: '-0Acceleration values (x'
Sensor data parsing error: not enough values to unpack (expected 3, got 1)
Sensor data parsing error: could not convert string to float: '-32.47Acceleration values (x'
Sensor data parsin

In [50]:
# 데이터 생성 센서 + pipe
import cv2
import mediapipe as mp
import numpy as np
import time, os, csv, serial

# MediaPipe 설정
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 시리얼 포트 설정
ser = serial.Serial('/dev/ttyACM6', 9600)

# 카메라 설정
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

# 동작 및 시간 설정
# actions = ['uf_outside', 'uf_inside',
#            'ur_outside', 'ur_inside', 'ur_above',
#            'ul_outside', 'ul_inside', 'ul_above',
#            'lf_outside', 'lf_inside',
#            'lr_outside', 'lr_inside', 'lr_above',
#            'll_outside', 'll_inside', 'll_avobe']
actions = ['ll_above', 'uf_outside']

seq_length = 30                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
secs_for_action = 30
created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

# 데이터 수집 및 저장
while cap.isOpened():
    for idx, action in enumerate(actions):
        combined_data = []  # 미디어파이프 데이터와 센서 데이터를 결합한 리스트

        ret, img = cap.read()
        img = cv2.flip(img, 1)
        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(7000)

        start_time = time.time()

        # 센서 데이터 초기값 설정
        last_gyro_x = last_gyro_y = last_gyro_z = last_acc_x = last_acc_y = last_acc_z = 0
        gyro_x = gyro_y = gyro_z = acc_x = acc_y = acc_z = 0

        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()
            img = cv2.flip(img, 1)
            out.write(img)  # 영상 저장

            # MediaPipe 처리
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # MediaPipe 데이터 처리
            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # 계산된 각도
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
                    v = v2 - v1
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
                    angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
                    angle = np.degrees(angle)

                    # 미디어파이프 데이터와 라벨
                    mediapipe_data_only = np.concatenate([joint.flatten(), angle])
                    action_label = idx

                    # 센서 데이터 처리
                    new_data_collected = False
                    if ser.in_waiting > 0:
                        ser_data = ser.readline().decode('utf-8').strip()

                        try:
                            if 'Gyroscope values' in ser_data:
                                gyro_values = ser_data.split(': ')[1].split(', ')
                                gyro_x, gyro_y, gyro_z = [float(val) for val in gyro_values]
                                new_data_collected = True

                            elif 'Acceleration values' in ser_data:
                                acc_values = ser_data.split(': ')[1].split(', ')
                                acc_x, acc_y, acc_z = [float(val) for val in acc_values]
                                new_data_collected = True

                        except ValueError as e:
                            print("Sensor data parsing error:", e)

                    # 센서 데이터가 수집되지 않았다면 이전 값을 사용
                    if not new_data_collected:
                        gyro_x, gyro_y, gyro_z = last_gyro_x, last_gyro_y, last_gyro_z
                        acc_x, acc_y, acc_z = last_acc_x, last_acc_y, last_acc_z

                    # 데이터 결합 및 저장
                    sensor_data = [gyro_x, gyro_y, gyro_z, acc_x, acc_y, acc_z]
                    combined_row = np.concatenate((mediapipe_data_only, sensor_data, [action_label]))
                    combined_data.append(combined_row)

                    # 마지막 센서값 갱신
                    last_gyro_x, last_gyro_y, last_gyro_z = gyro_x, gyro_y, gyro_z
                    last_acc_x, last_acc_y, last_acc_z = acc_x, acc_y, acc_z

            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break

        # 결합된 데이터 저장
        combined_data = np.array(combined_data)
        np.save(os.path.join('dataset', f'combined_{action}_{created_time}'), combined_data)

        # 시퀀스 데이터 생성 및 저장
        full_seq_data = []
        for seq in range(len(combined_data) - seq_length):
            full_seq_data.append(combined_data[seq:seq + seq_length])
        full_seq_data = np.array(full_seq_data)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)

    break

cap.release()
out.release()
cv2.destroyAllWindows()


I0000 00:00:1702910034.148736   19723 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702910034.196465   27842 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.54.03), renderer: NVIDIA RTX A5000/PCIe/SSE2
QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30

In [30]:
data = np.load('dataset/combined_lower_left_1702755196.npy')
data.shape

(445, 106)

In [9]:
import pandas as pd
data = np.load('dataset/seq_lower_left_1702802951.npy')
print(data.shape)
data = pd.DataFrame(data)
data

(868, 30, 106)


ValueError: Must pass 2-d input. shape=(868, 30, 106)

In [51]:
# 모델 판단
import cv2
import mediapipe as mp
import numpy as np
import serial
from tensorflow.keras.models import load_model

# MediaPipe 설정
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 시리얼 포트 설정
ser = serial.Serial('/dev/ttyACM6', 9600)

# 모델 불러오기
model = load_model('models/model.h5', compile=False)

# 카메라 설정
cap = cv2.VideoCapture(0)

# 동작 설정
# actions = ['uf_outside', 'uf_inside',
#            'ur_outside', 'ur_inside', 'ur_above',
#            'ul_outside', 'ul_inside', 'ul_above',
#            'lf_outside', 'lf_inside',
#            'lr_outside', 'lr_inside', 'lr_above',
#            'll_outside', 'll_inside', 'll_avobe']

actions = ['ll_avobe', 'uf_outside']

seq_length = 30

seq = []
last_sensor_data = [0, 0, 0, 0, 0, 0]  # 센서 데이터의 마지막 값을 저장하기 위한 변수

while cap.isOpened():
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # 관절 간 각도 계산
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
            angle = np.degrees(angle)

            d = np.concatenate([joint.flatten(), angle])

            # 센서 데이터 읽기 및 처리
            sensor_data = last_sensor_data.copy()  # 이전 센서 데이터 사용
            if ser.in_waiting > 0:
                ser_data = ser.readline().decode('utf-8').strip()

                try:
                    if 'Gyroscope values' in ser_data:
                        gyro_values = ser_data.split('Gyroscope values')[1].split(': ')[1].split(', ')
                        sensor_data[0:3] = [float(val) for val in gyro_values[:3]]

                    if 'Acceleration values' in ser_data:
                        acc_values = ser_data.split('Acceleration values')[1].split(': ')[1].split(', ')
                        sensor_data[3:6] = [float(val) for val in acc_values[:3]]

                    last_sensor_data = sensor_data  # 센서 데이터 업데이트

                except ValueError as e:
                    print("Sensor data parsing error:", e)
                    continue

            combined_data = np.concatenate((d, sensor_data))
            seq.append(combined_data)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            # 모델 예측
            if len(seq) == seq_length:
                input_data = np.expand_dims(np.array(seq, dtype=np.float32), axis=0)
                y_pred = model.predict(input_data).squeeze()
                i_pred = int(np.argmax(y_pred))
                conf = y_pred[i_pred]

                if conf > 0.9:
                    action = actions[i_pred]
                    cv2.putText(img, f'{action.upper()} ({conf:.2f})', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                del seq[0]

    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1702910252.756496   19723 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702910252.788651   29468 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.54.03), renderer: NVIDIA RTX A5000/PCIe/SSE2
QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30).
Cannot move to target thread (0x68f3fc0)

QObject::moveToThread: Current thread (0x68f3fc0) is not the object's thread (0x6b1ac30

1/1 [==============================] - 0s 21ms/step


In [4]:
# 모델판단 및 0.5초 유지된 동작 jason파일에 갱신
import cv2
import mediapipe as mp
import numpy as np
import serial
from tensorflow.keras.models import load_model
import json
import time

# MediaPipe 설정
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 시리얼 포트 설정
ser = serial.Serial('/dev/ttyACM6', 9600)

# 모델 불러오기
model = load_model('models/16_ll_above_.h5', compile=False)

# 카메라 설정
cap = cv2.VideoCapture(0)

# 동작 설정
actions = ['ll_avobe', 'uf_outside']

seq_length = 30
seq = []
last_sensor_data = [0, 0, 0, 0, 0, 0]  # 센서 데이터의 마지막 값을 저장하기 위한 변수

last_action = None
last_action_time = time.time()
action_duration = 0.5  # 같은 동작이 유지되어야 하는 시간 (초)

while cap.isOpened():
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # 관절 간 각도 계산
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
            angle = np.degrees(angle)

            d = np.concatenate([joint.flatten(), angle])

            # 센서 데이터 읽기 및 처리
            sensor_data = last_sensor_data.copy()  # 이전 센서 데이터 사용
            if ser.in_waiting > 0:
                ser_data = ser.readline().decode('utf-8').strip()

                try:
                    if 'Gyroscope values' in ser_data:
                        gyro_values = ser_data.split('Gyroscope values')[1].split(': ')[1].split(', ')
                        sensor_data[0:3] = [float(val) for val in gyro_values[:3]]

                    if 'Acceleration values' in ser_data:
                        acc_values = ser_data.split('Acceleration values')[1].split(': ')[1].split(', ')
                        sensor_data[3:6] = [float(val) for val in acc_values[:3]]

                    last_sensor_data = sensor_data  # 센서 데이터 업데이트

                except ValueError as e:
                    print("Sensor data parsing error:", e)
                    continue

            # 모델에 맞는 데이터 형태로 결합
            combined_data = np.concatenate((d, sensor_data))
            if len(combined_data) < 105:
                # 105개 요소가 되지 않을 경우 직전 센서값으로 채움
                combined_data = np.pad(combined_data, (0, 105 - len(combined_data)), 'constant', constant_values=last_sensor_data[-1])

            seq.append(combined_data)
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) == seq_length:
                input_data = np.expand_dims(np.array(seq, dtype=np.float32), axis=0)
                y_pred = model.predict(input_data).squeeze()
                i_pred = int(np.argmax(y_pred))
                conf = y_pred[i_pred]

                if conf > 0.9:
                    action = actions[i_pred]
                    cv2.putText(img, f'{action.upper()} ({conf:.2f})', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                    if action == last_action and (time.time() - last_action_time) > action_duration:
                        data = {"brushedSection": action, "brushedCnt": 1}
                        with open('action_data.json', 'w') as f:
                            json.dump(data, f)
                        last_action_time = time.time()

                    last_action = action
                del seq[0]

    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1702914915.463306    7962 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702914915.499180    8269 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.54.03), renderer: NVIDIA RTX A5000/PCIe/SSE2
QObject::moveToThread: Current thread (0x80457b0) is not the object's thread (0x82b01b0).
Cannot move to target thread (0x80457b0)

QObject::moveToThread: Current thread (0x80457b0) is not the object's thread (0x82b01b0).
Cannot move to target thread (0x80457b0)

QObject::moveToThread: Current thread (0x80457b0) is not the object's thread (0x82b01b0).
Cannot move to target thread (0x80457b0)

QObject::moveToThread: Current thread (0x80457b0) is not the object's thread (0x82b01b0).
Cannot move to target thread (0x80457b0)

QObject::moveToThread: Current thread (0x80457b0) is not the object's thread (0x82b01b0).
Cannot move to target thread (0x80457b0)

QObject::moveToThread: Current thread (0x80457b0) is not the object's thread (0x82b01b0

1/1 [==============================] - 0s 21ms/step


In [1]:
# 저장된 jason파일 서버에 전송
import cv2
import mediapipe as mp
import numpy as np
import serial
from tensorflow.keras.models import load_model
import json
import time
import requests

# MediaPipe 설정
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 시리얼 포트 설정
ser = serial.Serial('/dev/ttyACM5', 9600)

# 모델 불러오기
model = load_model('models/06_ul_outside_.h5', compile=False)

# 카메라 설정
cap = cv2.VideoCapture(0)

# 동작 설정
# actions = ['uf_outside', 'uf_inside',
#            'ur_outside', 'ur_inside', 'ur_above',
#            'ul_outside', 'ul_inside', 'ul_above',
#            'lf_outside', 'lf_inside',
#            'lr_outside', 'lr_inside', 'lr_above',
#            'll_outside', 'll_inside', 'll_avobe']

actions = ['ul_outside', 'll_avobe']

seq_length = 30

seq = []
action_seq = []
last_sensor_data = [0, 0, 0, 0, 0, 0]

last_action = None
last_action_time = time.time()
action_duration = 0.5  # 같은 동작이 유지되어야 하는 시간 (초)

url = "http://localhost:8080/api/teeth"  # 데이터를 전송할 URL

while cap.isOpened():
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # 관절 간 각도 계산
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
            angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
            angle = np.degrees(angle)

            d = np.concatenate([joint.flatten(), angle])

            # 센서 데이터 읽기 및 처리
            sensor_data = last_sensor_data.copy()  # 이전 센서 데이터 사용
            if ser.in_waiting > 0:
                ser_data = ser.readline().decode('utf-8').strip()

                try:
                    if 'Gyroscope values' in ser_data:
                        gyro_values = ser_data.split('Gyroscope values')[1].split(': ')[1].split(', ')
                        sensor_data[0:3] = [float(val) for val in gyro_values[:3]]

                    if 'Acceleration values' in ser_data:
                        acc_values = ser_data.split('Acceleration values')[1].split(': ')[1].split(', ')
                        sensor_data[3:6] = [float(val) for val in acc_values[:3]]

                    last_sensor_data = sensor_data

                except ValueError as e:
                    print("Sensor data parsing error:", e)
                    continue

            # 모델에 맞는 데이터 형태로 결합
            combined_data = np.concatenate((d, sensor_data))
            seq.append(combined_data)
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) == seq_length:
                input_data = np.expand_dims(np.array(seq, dtype=np.float32), axis=0)
                y_pred = model.predict(input_data).squeeze()
                i_pred = int(np.argmax(y_pred))
                conf = y_pred[i_pred]

                if conf > 0.9:
                    action = actions[i_pred]
                    cv2.putText(img, f'{action.upper()} ({conf:.2f})', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                    # 동작 유지 시간 체크 및 JSON 파일 저장
                    if action == last_action and (time.time() - last_action_time) >= action_duration:
                        # JSON 파일에 저장 및 데이터 전송
                        data = {"brushedSection": action, "brushedCnt": 1}
                        with open('action_data.json', 'w') as f:
                            json.dump(data, f)
                        last_action_time = time.time()

                        # 데이터를 localhost로 전송
                        response = requests.post(url, json=data)
                        print("Data sent, response:", response.text)

                    last_action = action

                del seq[0]

    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


2023-12-19 03:49:03.455059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 03:49:03.602022: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 03:49:03.607049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/piai/anaconda3/envs/opencv_keras/lib/python3.8/site-packages/cv2/../../lib64:
2023-12-19 0

1/1 [==============================] - 0s 25ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] - 0s 21ms/step
Data sent, response: 
1/1 [==============================] - 0s 21ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] - 0s 21ms/step
Data sent, response: 
1/1 [==============================] - 0s 21ms/step
Data sent, response: 
1/1 [==============================] - 0s 21ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] - 0s 22ms/step
Data sent, response: 
1/1 [==============================] -